<a href="https://colab.research.google.com/github/paulf35/winequality/blob/main/CD_Proj2_SpanishWines_ModelingV1_CORE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict the quality of Spanish wines based on existing data
Data Source: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset



# Add imports and functions

## Imports

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA


## Models
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

## Regression Metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


## Set global scikit-learn configuration
from sklearn import set_config
## Display estimators as a diagram
set_config(display='diagram') # 'text' or 'diagram'}

from IPython.core.display import clear_output

# Warnings
import warnings

## Set filter warnings to ignore
warnings.filterwarnings('ignore')

# Set the default transformation output to Pandas
from sklearn import set_config
set_config(transform_output='pandas')

## Functions

In [ ]:
# Explore Univariate Categorical Function
def explore_categorical(df, x, fillna = True, placeholder = 'MISSING',
                        figsize = (20,8), order = None):
  """Creates a seaborn countplot with the option to temporarily fill missing values
  Prints statements about null values, cardinality, and checks for
  constant/quasi-constant features.
  Source:{PASTE IN FINAL LESSON LINK}
  """
  # Make a copy of the dataframe and fillna
  temp_df = df.copy()
  # Before filling nulls, save null value counts and percent for printing
  null_count = temp_df[x].isna().sum()
  null_perc = null_count/len(temp_df)* 100
  # fillna with placeholder
  if fillna == True:
    temp_df[x] = temp_df[x].fillna(placeholder)
  # Create figure with desired figsize
  fig, ax = plt.subplots(figsize=figsize)
  # Plotting a count plot
  sns.countplot(data=temp_df, x=x, ax=ax, order=order)
  # Rotate Tick Labels for long names
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
  # Add a title with the feature name included
  ax.set_title(f"Column: {x}", fontweight='bold')

  # Fix layout and show plot (before print statements)
  fig.tight_layout()
  plt.show()

  # Print null value info
  print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")
  # Print cardinality info
  nunique = temp_df[x].nunique()
  print(f"- Unique Values: {nunique}")

  # First find value counts of feature
  val_counts = temp_df[x].value_counts(dropna=False)
  # Define the most common value
  most_common_val = val_counts.index[0]
  # Define the frequency of the most common value
  freq = val_counts.values[0]
  # Calculate the percentage of the most common value
  perc_most_common = freq / len(temp_df) * 100

  # Print the results
  print(f"- Most common value: '{most_common_val}' occurs {freq} times ({round(perc_most_common,2)}%)")
  # print message if quasi-constant or constant (most common val more than 98% of data)
  if perc_most_common > 98:
    print(f"\n- [!] Warning: '{x}' is a constant or quasi-constant feature and should be dropped.")
  else:
    print("- Not constant or quasi-constant.")
  return fig, ax

# Explore Univariate Numerical Function
def explore_numeric(df, x, figsize=(6,5) ):
  """Creates a seaborn histplot and boxplot with a share x-axis,
  Prints statements about null values, cardinality, and checks for
  constant/quasi-constant features.
  Source:{PASTE IN FINAL LESSON LINK}
  """

  ## Save null value counts and percent for printing
  null_count = df[x].isna().sum()
  null_perc = null_count/len(df)* 100


  ## Making our figure with gridspec for subplots
  gridspec = {'height_ratios':[0.7,0.3]}
  fig, axes = plt.subplots(nrows=2, figsize=figsize,
                           sharex=True, gridspec_kw=gridspec)
  # Histogram on Top
  sns.histplot(data=df, x=x, ax=axes[0])

  # Boxplot on Bottom
  sns.boxplot(data=df, x=x, ax=axes[1])

  ## Adding a title
  axes[0].set_title(f"Column: {x}", fontweight='bold')

  ## Adjusting subplots to best fill Figure
  fig.tight_layout()

  # Ensure plot is shown before message
  plt.show()


  # Print null value info
  print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")
  # Print cardinality info
  nunique = df[x].nunique()
  print(f"- Unique Values: {nunique}")


  # Get the most most common value, its count as # and as %
  most_common_val_count = df[x].value_counts(dropna=False).head(1)
  most_common_val = most_common_val_count.index[0]
  freq = most_common_val_count.values[0]
  perc_most_common = freq / len(df) * 100

  print(f"- Most common value: '{most_common_val}' occurs {freq} times ({round(perc_most_common,2)}%)")

  # print message if quasi-constant or constant (most common val more than 98% of data)
  if perc_most_common > 98:
    print(f"\n- [!] Warning: '{x}' is a constant or quasi-constant feature and should be dropped.")
  else:
    print("- Not constant or quasi-constant.")
  return fig, axes

  # Helper Function
# This custom function accept true targets and predictions with custom label
# Calculate and print  MAE, MSE , RMSE and R2 scores by saving it in a dictionary

def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics

# Helper Function
# This custom function accept the model, X_train, y_train, X_test, and y_test
# Obtain the predictions from the model for both training and test data
# Input the true and predicted values into the helper function to obtain all the metrics for both the training and test data.
# Print the results (optional with default as True
# Save the results as a dataframe (optional with default as False)

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False,model_name =''):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)

  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label= model_name + ' ' + 'Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label=model_name + ' ' + 'Test Data' )

  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

# Load data and mount Google Drive

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load data in from Google Drive
fname = "/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week07/Data/wines_SPA.csv"
df = pd.read_csv(fname)

# Load first 5 rows
df.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


# Data Exploration

In [ ]:
# Check types
df.dtypes

winery          object
wine            object
year            object
rating         float64
num_reviews      int64
country         object
region          object
price          float64
type            object
body           float64
acidity        float64
dtype: object

In [ ]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       7500 non-null   object 
 1   wine         7500 non-null   object 
 2   year         7498 non-null   object 
 3   rating       7500 non-null   float64
 4   num_reviews  7500 non-null   int64  
 5   country      7500 non-null   object 
 6   region       7500 non-null   object 
 7   price        7500 non-null   float64
 8   type         6955 non-null   object 
 9   body         6331 non-null   float64
 10  acidity      6331 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 644.7+ KB


There are missing vlaues for type, body, acidity, and year. We'll need to impute those before building the models.

In [ ]:
# Shape
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns.')

There are 7500 rows and 11 columns.


In [ ]:
# Describe
df.describe(exclude='number')

,winery,wine,year,country,region,type
count,7500,7500,7498,7500,7500,6955
unique,480,847,71,1,76,21
top,Contino,Reserva,2011,Espana,Rioja,Rioja Red
freq,457,467,1190,7500,2440,2357


Notes:
- No major outliers, but winery and wine are very high cardinality.

In [ ]:
df.describe(include='number')

,rating,num_reviews,price,body,acidity
count,7500.000000,7500.000000,7500.000000,6331.000000,6331.000000
mean,4.254933,451.109067,60.095822,4.158427,2.946612
std,0.118029,723.001856,150.356676,0.583352,0.248202
min,4.200000,25.000000,4.990000,2.000000,1.000000
25%,4.200000,389.000000,18.900000,4.000000,3.000000
50%,4.200000,404.000000,28.530000,4.000000,3.000000
75%,4.200000,415.000000,51.350000,5.000000,3.000000
max,4.900000,32624.000000,3119.080000,5.000000,3.000000


In [ ]:
# Check object columns for inconsistent data

# Create a series of the datatypes
d_types = df.dtypes
# Create a filter to select only the object datatypes
object_d_types = d_types[(d_types == "object")]
# Display the series of object datatypes
object_d_types

# display unique values from each column
for column in object_d_types.index:
  print(column)
  print(df[column].unique())
  print('\n')

winery
['Teso La Monja' 'Artadi' 'Vega Sicilia' 'Pago de Carraovejas'
 'Toro Albala' 'Bodegas El Nido' 'Valdespino' 'Dominio de Pingus'
 'Alvaro Palacios' 'Ordonez' 'Bodegas Valduero' 'Vina Sastre'
 'Sierra Cantabria' 'Descendientes de J. Palacios' 'La Rioja Alta'
 'Marques de Murrieta' 'Vinedos de Paganos' 'Emilio Moro'
 'Quinta de la Quietud' 'Bodegas Mauro' 'Bodega Contador (Benjamin Romeo)'
 'Remirez de Ganuza' 'Bodegas San Roman' 'Pago de Los Capellanes'
 'Bodega Numanthia' 'Alto Moncayo' 'Mas Doix' 'Finca Moncloa'
 'Bodegas Roda' 'Martinet' 'Recaredo' 'Clos Erasmus' 'Barbadillo'
 'Gonzalez-Byass' 'Bodegas Amaren' 'Alvear' 'Equipo Navazos' 'Morca'
 'Territorio Luthier' 'Rafael Palacios' 'Terra Remota'
 'Dehesa de Los Canonigos' 'Miguel Merino' 'Gutierrez de la Vega' 'Alion'
 'Aalto' 'Carmelo Rodero' 'Dominio del Bendito' "Mas d'en Gil"
 'Casa Castillo' 'Matarromera' 'Nin-Ortiz' 'Vinas del Vero'
 'Marques de Riscal' 'Arzuaga' 'Bodegas Mas Alta' 'Dominio de Calogia'
 'Tomas Postigo'

In [ ]:
# Check number columns for inconsistent data

# Create a series of the datatypes
d_types = df.dtypes
# Create a filter to select only the object datatypes
num_d_types = d_types[(d_types != "object")]
# Display the series of object datatypes
num_d_types

# display unique values from each column
for column in num_d_types.index:
  print(column)
  print(df[column].unique())
  print('\n')

rating
[4.9 4.8 4.7 4.6 4.5 4.4 4.3 4.2]


num_reviews
[   58    31  1793  1705  1309  1209  1201   926   643   630   591   454
   438   417   398   372   295   250   217   211   174   172   145   139
   125   118   103    87    84    79    70    69    68    56    52    50
    40    32    28    26 12421  5266  4350  3929  3437  3164  3127  2935
  2826  2765  2480  2419  2177  1892  1199  1141   940   935   690   685
   675   593   560   543   511   476   442   425   393   347   312   308
   303   300   292   291   285   280   268   264   256   254   251   243
   240   225   220   214   205   203   196   184   173   171   137   136
   134   133   132   131   127   123   117   116   112   110   108   102
   100    94    92    89    82    80    74    72    67    65    64    63
    60    59    57    53    51    49    47    45    42    41    39    37
    35    33    30    29    27  6803  5938  5545  5116  4747  4685  4516
  3383  3239  2416  2208  1608  1363  1108  1015   936   840   790   

In [ ]:
## Display the number of duplicate rows in the dataset
print(f'There are {df.duplicated().sum()} duplicate rows.')

There are 5452 duplicate rows.


## Summary of Cleanup Needed
- Remove all duplicate rows
- Remove `winery`, `region`, and `wine` columns because of high cardinality in data.  
- Address `N.V.` value in Year feature
- Remove country because they are all labeled as `Espana'


# Data Cleanup

## Remove duplicate rows

In [ ]:
# Remove duplicate rows
df = df.drop_duplicates()

## Confirm duplicate rows have been dropped
print(f'There are {df.duplicated().sum()} duplicate rows.')


There are 0 duplicate rows.


## Clean up inconsistent 'year' values

In [ ]:
# Remove rows with missing columns
df.dropna(subset=['year'], inplace=True)

#Remove N.V. rows from year
df = df[df.year != 'N.V.']

## Drop columns

In [ ]:
# Remove unneeded columns
df.drop(columns=['country','winery', 'wine', 'region'], inplace=True)

## Verify changes

In [ ]:
#Verify changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 6100
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         1976 non-null   object 
 1   rating       1976 non-null   float64
 2   num_reviews  1976 non-null   int64  
 3   price        1976 non-null   float64
 4   type         1877 non-null   object 
 5   body         1714 non-null   float64
 6   acidity      1714 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 123.5+ KB


# Preprocessing

## Determine Ordinal Features

Oridinal features: There are no ordinal features.

## Split Data

In [ ]:
# split X and y, we are predicting price
X = df.drop(columns= 'price')
y = df['price']

# split training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Create Pipelins and Tuples for Group of Columns

- Numeric features: rating, num_reviews, price (target), body, acidity
- Nominal features: region, type, year

### Numeric Pipeline

In [ ]:
# PREPROCESSING PIPELINE FOR NUMERIC DATA

# Save list of number column names
num_cols = X_train.select_dtypes('number').columns

# Transformers
scaler = StandardScaler()
med_imputer = SimpleImputer(strategy = 'median')

# Pipeline
num_pipeline = make_pipeline(med_imputer, scaler)
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [ ]:
# Numeric Tuple
numeric_tuple = ('number',num_pipeline, num_cols)

### Nominal Pipeline

In [ ]:
# PREPROCESSING PIPELINE FOR ONE-HOT-ENCODED DATA

# Save list of nominal column names
nominal_cols = X_train.select_dtypes('object').columns

# Transformers

missing_imputer = SimpleImputer(strategy='constant', fill_value='missing')
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Pipeline
nom_pipeline = make_pipeline(missing_imputer, ohe)
nom_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [ ]:
# Nominal Tuple
ohe_tuple = ('categorical',nom_pipeline, nominal_cols)

### Create Column Transformer

In [ ]:
# Instantiate the make column transformer
col_transformer = ColumnTransformer([numeric_tuple,
                                       ohe_tuple],
                                       remainder='drop', verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('number',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['year', 'type'], dtype='object'))],
                  verbose_feature_names_out=False)

# Bagged Trees Modeling

## Default Bagged Trees Model

In [ ]:
# Instantiate Bagged Tree Regressor
bagreg = BaggingRegressor(random_state = 42)

# Model Pipeline
bagreg_pipe = make_pipeline(col_transformer, bagreg)

# Fit the model pipeline on the training data only
bagreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('number',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['year', 'type'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('baggingregressor', BaggingRegressor(random_state=42))])

In [ ]:
# Use custom function to evaluate default model
evaluate_regression(bagreg_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics:  Training Data
------------------------------------------------------------
- MAE = 36.379
- MSE = 10,838.547
- RMSE = 104.108
- R^2 = 0.868

------------------------------------------------------------
Regression Metrics:  Test Data
------------------------------------------------------------
- MAE = 84.044
- MSE = 52,347.817
- RMSE = 228.796
- R^2 = 0.126


## Tuned Bagged Trees Regression Model

In [ ]:
# Get Parameters for tuning
bagreg_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('number',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                   ('categorical',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(fill_value='missing',
                                                                   strategy='constant')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                 

In [ ]:
# Define parameters to tune
params_bt = {'baggingregressor__n_estimators': [5, 10, 20, 30, 35, 40, 45, 50],
              'baggingregressor__max_samples' : [.5, .7, .8, .9 ],
              'baggingregressor__max_features': [.5, .7, .9, .95 ]}
# Instaniate the gridsearch
gridsearch = GridSearchCV(bagreg_pipe, params_bt, n_jobs=-1, verbose=1)
# Fit the gridsearch on the training data
gridsearch.fit(X_train, y_train)


Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('number',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='miss...
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['year', 'type'], dtype='object'))],
                                                          verbose_feature_names_out=False)),
                                       ('baggingregressor',
                                        BaggingRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'baggingregressor__max_features': [0.5, 0.7, 0.9,
                                                            0.95],
                         'baggingregressor__max_samples': [0.5, 0.7, 0.8, 0.9],
                         'baggingregressor__n_estimators': [5, 10, 20, 30, 35,
                                                            40, 45, 50]},
             verbose=1)

In [ ]:
# Obtain the best paramters from the gridsearch
gridsearch.best_params_

{'baggingregressor__max_features': 0.7,
 'baggingregressor__max_samples': 0.5,
 'baggingregressor__n_estimators': 50}

In [ ]:
best_bagreg = gridsearch.best_estimator_

In [ ]:
# Use custom function to evaluate default model
evaluate_regression(best_bagreg, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics:  Training Data
------------------------------------------------------------
- MAE = 63.399
- MSE = 23,246.337
- RMSE = 152.467
- R^2 = 0.716

------------------------------------------------------------
Regression Metrics:  Test Data
------------------------------------------------------------
- MAE = 79.766
- MSE = 40,032.175
- RMSE = 200.080
- R^2 = 0.331


# KNN Regressor Model


## Default KNN Regressor model

In [ ]:
# Instantiate Bagged Tree Regressor
knreg = KNeighborsRegressor()

# Model Pipeline
knreg_pipe = make_pipeline(col_transformer, knreg)

# Fit the model pipeline on the training data only
knreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('number',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['year', 'type'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('kneighborsregressor', KNeighborsRegressor())])

In [ ]:
# Use custom function to evaluate default model
evaluate_regression(knreg_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics:  Training Data
------------------------------------------------------------
- MAE = 71.990
- MSE = 33,242.756
- RMSE = 182.326
- R^2 = 0.594

------------------------------------------------------------
Regression Metrics:  Test Data
------------------------------------------------------------
- MAE = 80.234
- MSE = 44,633.563
- RMSE = 211.267
- R^2 = 0.255


##Tuned KNN Regressor Model

In [ ]:
# Get Parameters for tuning
knreg_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('number',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                   ('categorical',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(fill_value='missing',
                                                                   strategy='constant')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                 

In [ ]:
# Define parameters to tune
params_kn = {'kneighborsregressor__leaf_size': [5, 10, 20, 30, 35, 40],
              'kneighborsregressor__n_neighbors' : [1, 5, 10, 15]}
# Instaniate the gridsearch
gridsearch = GridSearchCV(knreg_pipe, params_kn, verbose=1)
# Fit the gridsearch on the training data
gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('number',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['rating', 'num_reviews', 'body', 'acidity'], dtype='object')),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['year', 'type'], dtype='object'))],
                                                          verbose_feature_names_out=False)),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__leaf_size': [5, 10, 20, 30, 35,
                                                            40],
                         'kneighborsregressor__n_neighbors': [1, 5, 10, 15]},
             verbose=1)

In [ ]:
# Obtain the best paramters from the gridsearch
gridsearch.best_params_

{'kneighborsregressor__leaf_size': 5, 'kneighborsregressor__n_neighbors': 15}

In [ ]:
best_knreg = gridsearch.best_estimator_

In [ ]:
# Use custom function to evaluate default model
evaluate_regression(best_knreg, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics:  Training Data
------------------------------------------------------------
- MAE = 78.326
- MSE = 38,111.295
- RMSE = 195.221
- R^2 = 0.534

------------------------------------------------------------
Regression Metrics:  Test Data
------------------------------------------------------------
- MAE = 74.746
- MSE = 38,466.114
- RMSE = 196.128
- R^2 = 0.358
